In [458]:
import json
import pandas as pd
from datetime import datetime
import os
import requests
from bs4 import BeautifulSoup
import time
from random import random

In [ ]:
with open('first_100_rows.json') as f:
    reviews = []
    for row in f:
        reviews.append(json.loads(row))

In [459]:
goodreads_key = os.environ['GOODREADS_API_KEY']
goodreads_secret = os.environ['GOODREADS_SECRET']
aws_key = os.environ['AWS_ACCESS_KEY_ID']
aws_secret = os.environ['AWS_SECRET_ACCESS_KEY']

In [ ]:
df = pd.DataFrame(reviews)

In [ ]:
reviews

In [ ]:
df

In [ ]:
df['asin'][26]

In [ ]:
with open('combined_lists.json') as f:
    contents = json.load(f)
    

In [ ]:
contents

In [ ]:
flat_data = []
for list_date, lst in contents.iteritems():
    for title, meta in lst.iteritems():
        d = meta.copy()
        d['title'] = title
        d['list_date'] = list_date
        flat_data.append(d)

In [ ]:
books = pd.DataFrame(flat_data)
# books = pd.read_csv('combined_bestseller_lists.csv', index_col = 0)

In [ ]:
books

In [ ]:
# books.to_csv('combined_bestseller_lists.csv', encoding='utf-8')

In [ ]:
books[books.isbn13.str.contains('B00LL0DR94')]

In [ ]:
len(books.title.unique())

In [ ]:
B00547KH66

In [ ]:
# books[books.isbn13.str.contains('A00')]
books[books['isbn13'].str.contains("978")]

In [ ]:
def get_isbns(books_df):
    for row in books_df.index:
        old_isbn10 = books.loc[row, 'isbn10']
        old_isbn13 = books.loc[row, 'isbn13']
        if old_isbn13.startswith('A00'):
            t = '+'.join(books.loc[row, 'title'].split())
            a = '+'.join(books.loc[row, 'author'].split())
#                 print t, a, row
            gr_isbn_url = 'https://www.goodreads.com/book/title.xml?author={}&key={}&title={}'.format(a, goodreads_key, t)
            gr_content = requests.get(gr_isbn_url)
            soup = BeautifulSoup(gr_content.content, "html.parser")
            try:
                new_isbn10 = soup.find('isbn').contents[0]
                new_isbn13 = soup.find('isbn13').contents[0]
                books_df.replace(to_replace = old_isbn10, value = new_isbn10, inplace = True)
                books_df.replace(to_replace = old_isbn13, value = new_isbn13, inplace = True)
            except AttributeError:
                new_isbn10 = old_isbn10
                new_isbn13 = old_isbn13

    return books_df

In [ ]:
get_isbns(books)

In [ ]:
books['isbn10'][2]

In [ ]:
a = '+'.join('Chris Culver')
t = '+'.join('the abbey')
gr_isbn_url = 'https://www.goodreads.com/book/title.xml?author={}&key={}&title={}'.format(a, goodreads_key, t)
gr_content = requests.get(gr_isbn_url)
soup = BeautifulSoup(gr_content.content, "html.parser")
str(soup)

In [ ]:
books.isbn13.replace(to_replace='A00B00547KH66', value='9780451198518', inplace = True)

In [ ]:
t = '+'.join(books.)

In [ ]:
'+'.join(books.loc[0, 'author'].split())

In [ ]:
books_copied = books.copy()

In [ ]:
books_copied.to_csv('books_copied.csv', encoding = 'utf-8')

In [ ]:
sorted(books.list_date.unique())

In [ ]:
one_list = books[books.list_date == '2011-02-26']

In [ ]:
one_list

In [461]:
pwd

u'/Users/asalmitrofanov/DSI/fiction-market-analysis-capstone'

In [464]:
books = pd.read_csv('data/book_meta_with_isbns.tsv', delimiter = '\t')

In [478]:
missing_isbns = books[(books['isbn10'] == ' ') & (books['isbn13'] == ' ') ]['author_title']

In [480]:
missing_isbns = pd.DataFrame(np.unique(missing_isbns.values), columns = ['author_title'])

In [483]:
missing_isbns['isbn10'] = None
missing_isbns['isbn13'] = None

In [493]:
missing_isbns[0]

TypeError: unhashable type

In [661]:
def get_isbns_google(books):
    isbns = {}
    books_not_found = []
    for book in books:
#         print book
        t = '+'.join(book.split('**')[1].replace("'", '').encode('utf-8').split())
        a = '+'.join(book.split('**')[0].replace('.', '').encode('utf-8').split())
        ggl_url = "https://www.google.com/search?q={}+{}+isbn&oq={}+{}+isbn&aqs=chrome.\
                    .69i57.11282j0j8&sourceid=chrome&ie=UTF-8".format(t, a, t, a)
        ggl_cont = requests.get(ggl_url)
        soup = BeautifulSoup(ggl_cont.content, 'html.parser')
        try:
            isbn = soup.findAll("span", {"class": "_G0d"})[0].text
            if len(isbn) == 13:
                isbns[book] = {'isbn10' : None,
                               'isbn13' : isbn}
            elif len(isbn) == 10:
                isbns[book] = {'isbn10' : isbn,
                               'isbn13' : None}
            else:
                isbns[book] = {'isbn10' : None,
                               'isbn13' : None}
        except IndexError:
            books_not_found.append(book)
            isbns[book] = {'isbn10' : None,
                           'isbn13' : None}
        time.sleep(1 + random()*3)
    return isbns, books_not_found

In [528]:
isbns_found, still_missing = get_isbns_google(missing_isbns.author_title.values)

In [539]:
isbns = pd.DataFrame(isbns_found).T

In [542]:
isbns.reset_index(inplace = True)

In [549]:
google_isbns = isbns[isbns.isbn13.notnull()]

In [553]:
google_isbns.to_csv('data/google_isbns.tsv', sep = '\t')

In [552]:
pwd

u'/Users/asalmitrofanov/DSI/fiction-market-analysis-capstone'

In [ ]:
different_format = []
no_list = []
parsed_pdf_flat_data = []
for list_date, lst in pdf_contents.iteritems():
    if lst == "This list is in a different format":
        different_format.append(list_date)
    elif lst == "No list on this date":
        no_list.append(list_date)
    else:
        for title, meta in lst.iteritems():
            d = meta.copy()
            d['title'] = title
            d['list_date'] = list_date
            parsed_pdf_flat_data.append(d)

In [ ]:
# different_format

In [ ]:
parsed_pdf_lists = pd.DataFrame(parsed_pdf_flat_data)

In [ ]:
asins_df = parsed_pdf_lists.copy()


In [ ]:
asins_df.insert(0, 'asin', None)

In [ ]:
asins_df['author_title'] = asins_df.author + '**' + asins_df.title

In [ ]:
asins_df.author_title.unique()

In [ ]:
len(parsed_pdf_lists.list_date.unique())

In [601]:
def get_isbns_asins(books):
    scraped = {}
    for book in books:
        isbn10 = None
        isbn13 = None
        asin = None
        book = book.encode('utf-8')
        t = '+'.join(book.split('**')[1].split())
        a = '+'.join(book.split('**')[0].split())

        gr_isbn_url = 'https://www.goodreads.com/book/title.xml?author={}&key={}&title={}'.format(a, goodreads_key, t)
        gr_content = requests.get(gr_isbn_url)
        soup = BeautifulSoup(gr_content.content, "html.parser")
        try:
            isbn10 = soup.find('isbn').contents[0]
            isbn13 = soup.find('isbn13').contents[0]
            asin = soup.find('kindle_asin').contents[0]
        except AttributeError:
            continue
        except UnicodeEncodeError:
            continue
        scraped[book] = {'isbn10':isbn10, 'isbn13':isbn13, 'asin':asin}
        time.sleep(1 + random()*2)

    with open('data/format2_books_with_isbns_part3.json', mode='w') as f:
        f.write(json.dumps(scraped, indent=2))

In [ ]:
get_isbns_asins(books)

In [ ]:
books = asins_df.author_title.unique()
for book in books:
    print book

In [ ]:
with open('data/books_with_isbns.json', mode='r') as f:
    book_data = json.load(f)

In [ ]:
book_data

In [567]:
with open('data/parsed_pdf_lists_format2.json', 'r') as f:
    scraped_books_format2 = json.load(f)
#     scraped_books_format2 = []
#     for row in f: 
#         print row    
#         scraped_books_format2.append(json.loads(row))

In [569]:
scraped_books_format2

{u'1979-06-24': {u'9 Other Top-ranking NATO Generals and Advisors.': {u'asin': None,
   u'author': u'9 Other Top-ranking NATO Generals and Advisors.',
   u'isbn10': None,
   u'isbn13': None,
   u'rank_last_wk': u'4',
   u'rank_this_wk': u'5',
   u'wks_on_list': u'9'},
  u'CHESAPEAKE': {u'asin': None,
   u'author': u' James A. Michener.',
   u'isbn10': None,
   u'isbn13': None,
   u'rank_last_wk': u'8',
   u'rank_this_wk': u'11',
   u'wks_on_list': u'45'},
  u'CLASS REUNION': {u'asin': None,
   u'author': u' Rona Jaffe.',
   u'isbn10': None,
   u'isbn13': None,
   u'rank_last_wk': u'11',
   u'rank_this_wk': u'14',
   u'wks_on_list': u'5'},
  u'GHOST STORY': {u'asin': None,
   u'author': u' Peter Straub.',
   u'isbn10': None,
   u'isbn13': None,
   u'rank_last_wk': u'12',
   u'rank_this_wk': u'12',
   u'wks_on_list': u'11'},
  u'GOOD AS GOLD': {u'asin': None,
   u'author': u' Joseph Heller.',
   u'isbn10': None,
   u'isbn13': None,
   u'rank_last_wk': u'10',
   u'rank_this_wk': u'7',
   

In [572]:
flat_format_2_data = []
for list_date, lst in scraped_books_format2.iteritems():
#     print lst
    if lst == 'No list on this date':
        continue
    else:
        for title, meta in lst.iteritems():
            d = meta.copy()
            d['title'] = title
            d['list_date'] = list_date
            flat_format_2_data.append(d)

In [574]:
scraped_format2_df = pd.DataFrame(flat_format_2_data)

In [576]:
scraped_format2_df['title'].replace(to_replace='9 Other Top-ranking NATO Generals and Advisors.', 
                                    value = 'THE THIRD WORLD WAR: August 1985', inplace = True)

In [579]:
scraped_format2_df['author'].replace(to_replace = '9 Other Top-ranking NATO Generals and Advisors.', 
                                     value = 'General Sir John Hackett and Other Top-ranking NATO Generals and Advisors.',
                                     inplace = True)

In [581]:
scraped_format2_df['author_title'] = scraped_format2_df['author'] + '**' + scraped_format2_df['title']

In [582]:
scraped_format2_df

,asin,author,isbn10,isbn13,list_date,rank_last_wk,rank_this_wk,title,wks_on_list,author_title
0,None,General Sir John Hackett and Other Top-ranking...,None,None,1979-06-24,4,5,THE THIRD WORLD WAR: August 1985,9,General Sir John Hackett and Other Top-ranking...
1,None,Robin Cook.,None,None,1979-06-24,13,8,SPHINX,2,Robin Cook.**SPHINX
2,None,Peter Straub.,None,None,1979-06-24,12,12,GHOST STORY,11,Peter Straub.**GHOST STORY
3,None,Joseph Heller.,None,None,1979-06-24,10,7,GOOD AS GOLD,16,Joseph Heller.**GOOD AS GOLD
4,None,James A. Michener.,None,None,1979-06-24,8,11,CHESAPEAKE,45,James A. Michener.**CHESAPEAKE
5,None,Herman Wouk.,None,None,1979-06-24,6,3,WAR AND REMEMBRANCE,38,Herman Wouk.**WAR AND REMEMBRANCE
6,None,Len Deighton.,None,None,1979-06-24,0,13,SS-GB,15,Len Deighton.**SS-GB
7,None,Walter F. Murphy.,None,None,1979-06-24,15,10,THE VICAR OF CHRIST,8,Walter F. Murphy.**THE VICAR OF CHRIST
8,None,Trevanian.,None,None,1979-06-24,3,6,SHIBUMI,8,Trevanian.**SHIBUMI
9,None,Richard Bach.,None,None,1979-06-24,7,15,THERE'S NO SUCH PLACE AS FAR AWAY,3,Richard Bach.**THERE'S NO SUCH PLACE AS FAR AWAY


In [587]:
books_to_get_isbns = scraped_format2_df.author_title.unique()

In [596]:
books_to_get_isbns

array([ 'General Sir John Hackett and Other Top-ranking NATO Generals and Advisors.**THE THIRD WORLD WAR: August 1985',
       u' Robin Cook.**SPHINX', u' Peter Straub.**GHOST STORY',
       u' Joseph Heller.**GOOD AS GOLD',
       u' James A. Michener.**CHESAPEAKE',
       u' Herman Wouk.**WAR AND REMEMBRANCE', u' Len Deighton.**SS-GB',
       u' Walter F. Murphy.**THE VICAR OF CHRIST', u' Trevanian.**SHIBUMI',
       u" Richard Bach.**THERE'S NO SUCH PLACE AS FAR AWAY"], dtype=object)

In [598]:
get_isbns_asins(books_to_get_isbns[:1000])

In [600]:
get_isbns_asins(books_to_get_isbns[1000:2000])

In [602]:
get_isbns_asins(books_to_get_isbns[2000:])

In [606]:
isbns = pd.read_json('data/format2_books_with_isbns.json').T

In [608]:
isbns.reset_index(inplace = True)

In [610]:
isbns

,index,asin,isbn10,isbn13
0,Dale Brown**DAY OF THE CHEETAH,B00IT7GKF0,0425120430,9780425120439
1,Danielle Steel**DADDY,B001TLZF8Q,9953299005,9789953299006
2,Helen MacInnes**PRELUDE TO TERROR,B00F1W07XM,0449213781,9780449213780
3,Jack Higgins**COLD HARBOUR,B000OIZV4I,0425193209,9780425193204
4,James A. Michener**CARIBBEAN,B00FO60AZA,0812974921,9780812974928
5,Jimmy Buffett**TALES FROM MARGARITAVILLE,,0156026988,9780156026987
6,Judith Michael**A RULING PASSION,,0671899589,9780671899585
7,Len Deighton**SPY LINE,B003Z6QGJK,0345420179,9780345420176
8,Peter Straub**MYSTERY,B000FC1CK0,0375756701,9780375756702
9,Phillip Margolin**PROOF POSITIVE,B000GCFWXC,0060735058,9780060735050


In [611]:
isbns2 = pd.read_json('data/format2_books_with_isbns_part2.json').T

In [612]:
isbns2.reset_index(inplace=True)

In [613]:
isbns2

,index,asin,isbn10,isbn13
0,Dan Brown**THE DA VINCI CODE,B000FA675C,0307277674,9780307277671
1,William F. Buckley Jr.**MARCO POLO,B0116EBXP4,1888952113,9781888952117
2,William F. Buckley Jr.**SEE YOU LATER ALLIGATOR,,1581825374,9781581825374
3,William F. Buckley Jr.**THE STORY OF HENRI TOD,,1888952121,9781888952124
4,**THE CLINIC,B000FBFN92,0553712802,9780553712803
5,1 Gates Jr.**1 Gates Jr.,,9780393059,
6,27**27,,0615974058,9780615974057
7,33**33,B000O76O8G,1591841038,9781591841036
8,MacDonald**MacDonald,,0064401480,9780064401487


In [614]:
isbns3 = pd.read_json('data/format2_books_with_isbns_part3.json').T

In [615]:
isbns3.reset_index(inplace = True)

In [616]:
isbns3

,index,asin,isbn10,isbn13
0,Annie Dillard**THE MAYTREES,B000ROKXZ6,0061239534,9780061239533
1,Graeme Base**THE ELEVENTH HOUR,,0810932652,9780810932654
2,Julian Barnes**ARTHUR & GEORGE,B01E8XKCY8,1400097037,9781400097036
3,Pat Conroy**THE PRINCE OF TIDES,B003XKN65K,0553381547,9780553381542
4,William F. Buckley Jr.**MONGOOSE R.I.P.,B0116EBXLI,0440202310,9780440202318
5,**THE THIRD DEADLY SIN,B00B4V7T0I,042510429X,9780425104293


In [622]:
isbns_1974_2008 = pd.concat((isbns, isbns2, isbns3), axis = 0, ignore_index=True)

In [624]:
isbns_1974_2008.rename(columns = lambda x: x.replace('index', 'author_title'), inplace = True)

In [625]:
isbns_1974_2008

,author_title,asin,isbn10,isbn13
0,Dale Brown**DAY OF THE CHEETAH,B00IT7GKF0,0425120430,9780425120439
1,Danielle Steel**DADDY,B001TLZF8Q,9953299005,9789953299006
2,Helen MacInnes**PRELUDE TO TERROR,B00F1W07XM,0449213781,9780449213780
3,Jack Higgins**COLD HARBOUR,B000OIZV4I,0425193209,9780425193204
4,James A. Michener**CARIBBEAN,B00FO60AZA,0812974921,9780812974928
5,Jimmy Buffett**TALES FROM MARGARITAVILLE,,0156026988,9780156026987
6,Judith Michael**A RULING PASSION,,0671899589,9780671899585
7,Len Deighton**SPY LINE,B003Z6QGJK,0345420179,9780345420176
8,Peter Straub**MYSTERY,B000FC1CK0,0375756701,9780375756702
9,Phillip Margolin**PROOF POSITIVE,B000GCFWXC,0060735058,9780060735050


In [642]:
books_1974_2008 = pd.merge(scraped_format2_df, isbns_1974_2008, how = 'left', on = 'author_title')

In [637]:
pd.merge?

In [647]:
books_1974_2008.drop(['asin_x', 'isbn10_x', 'isbn13_x'], axis = 1, inplace = True)

In [649]:
books_1974_2008.rename(columns = lambda x: x.replace('_y', ''), inplace = True)

In [651]:
isbns_still_missing = books_1974_2008[books_1974_2008.isbn13.isnull()]

In [659]:
isbns_needed = isbns_still_missing.author_title.unique()

In [662]:
isbns_found1, isbns_still_missing1 = get_isbns_google(isbns_needed)

In [671]:
google_isbns_found = pd.DataFrame(isbns_found1).T

In [673]:
google_isbns_found.reset_index(inplace = True)

In [677]:
google_isbns_found.rename(columns = lambda x: x.replace('index', 'author_title'), inplace = True)

In [679]:
google_isbns_found.to_csv('data/google_isbns_1974_to_2008.tsv', sep = '\t', encoding = 'utf-8')

In [668]:
books_1974_2008

,author,list_date,rank_last_wk,rank_this_wk,title,wks_on_list,author_title,asin,isbn10,isbn13
0,General Sir John Hackett and Other Top-ranking...,1979-06-24,4,5,THE THIRD WORLD WAR: August 1985,9,General Sir John Hackett and Other Top-ranking...,NaN,NaN,NaN
1,Robin Cook.,1979-06-24,13,8,SPHINX,2,Robin Cook.**SPHINX,NaN,NaN,NaN
2,Peter Straub.,1979-06-24,12,12,GHOST STORY,11,Peter Straub.**GHOST STORY,NaN,NaN,NaN
3,Joseph Heller.,1979-06-24,10,7,GOOD AS GOLD,16,Joseph Heller.**GOOD AS GOLD,NaN,NaN,NaN
4,James A. Michener.,1979-06-24,8,11,CHESAPEAKE,45,James A. Michener.**CHESAPEAKE,NaN,NaN,NaN
5,Herman Wouk.,1979-06-24,6,3,WAR AND REMEMBRANCE,38,Herman Wouk.**WAR AND REMEMBRANCE,NaN,NaN,NaN
6,Len Deighton.,1979-06-24,0,13,SS-GB,15,Len Deighton.**SS-GB,NaN,NaN,NaN
7,Walter F. Murphy.,1979-06-24,15,10,THE VICAR OF CHRIST,8,Walter F. Murphy.**THE VICAR OF CHRIST,NaN,NaN,NaN
8,Trevanian.,1979-06-24,3,6,SHIBUMI,8,Trevanian.**SHIBUMI,NaN,NaN,NaN
9,Richard Bach.,1979-06-24,7,15,THERE'S NO SUCH PLACE AS FAR AWAY,3,Richard Bach.**THERE'S NO SUCH PLACE AS FAR AWAY,NaN,NaN,NaN


In [689]:
books1 = pd.merge(isbns_still_missing, google_isbns_found, how = 'left', on = 'author_title')

In [ ]:
books1.drop(['isbn10_x', 'isbn13_x'], axis = 1, inplace = True)

In [696]:
books1.rename(columns = lambda x: x.replace('_y', ''), inplace = True)
books1 = books1.reindex_axis(sorted(books1.columns), axis = 1)

In [697]:
books1

,asin,author,author_title,isbn10,isbn13,list_date,rank_last_wk,rank_this_wk,title,wks_on_list
0,NaN,General Sir John Hackett and Other Top-ranking...,General Sir John Hackett and Other Top-ranking...,None,None,1979-06-24,4,5,THE THIRD WORLD WAR: August 1985,9
1,NaN,Robin Cook.,Robin Cook.**SPHINX,None,9788422612742,1979-06-24,13,8,SPHINX,2
2,NaN,Peter Straub.,Peter Straub.**GHOST STORY,None,9780965723138,1979-06-24,12,12,GHOST STORY,11
3,NaN,Joseph Heller.,Joseph Heller.**GOOD AS GOLD,None,9788071455707,1979-06-24,10,7,GOOD AS GOLD,16
4,NaN,James A. Michener.,James A. Michener.**CHESAPEAKE,None,9782020053662,1979-06-24,8,11,CHESAPEAKE,45
5,NaN,Herman Wouk.,Herman Wouk.**WAR AND REMEMBRANCE,None,9780316172141,1979-06-24,6,3,WAR AND REMEMBRANCE,38
6,NaN,Len Deighton.,Len Deighton.**SS-GB,None,9780745134116,1979-06-24,0,13,SS-GB,15
7,NaN,Walter F. Murphy.,Walter F. Murphy.**THE VICAR OF CHRIST,None,9780345320391,1979-06-24,15,10,THE VICAR OF CHRIST,8
8,NaN,Trevanian.,Trevanian.**SHIBUMI,None,9780345301390,1979-06-24,3,6,SHIBUMI,8
9,NaN,Richard Bach.,Richard Bach.**THERE'S NO SUCH PLACE AS FAR AWAY,None,9780002240215,1979-06-24,7,15,THERE'S NO SUCH PLACE AS FAR AWAY,3


In [ ]:
for book in books1.author_title.unique():
    

In [712]:
books_1974_2008[books_1974_2008.author_title == books1.author_title.unique()[4]]

,author,list_date,rank_last_wk,rank_this_wk,title,wks_on_list,author_title,asin,isbn10,isbn13
4,James A. Michener.,1979-06-24,8,11,CHESAPEAKE,45,James A. Michener.**CHESAPEAKE,NaN,NaN,NaN
91,James A. Michener.,1978-07-09,0,1,CHESAPEAKE,1,James A. Michener.**CHESAPEAKE,NaN,NaN,NaN
238,James A. Michener.,1979-01-14,1,1,CHESAPEAKE,22,James A. Michener.**CHESAPEAKE,NaN,NaN,NaN
852,James A. Michener.,1979-03-04,1,2,CHESAPEAKE,29,James A. Michener.**CHESAPEAKE,NaN,NaN,NaN
1143,James A. Michener.,1979-02-25,1,1,CHESAPEAKE,28,James A. Michener.**CHESAPEAKE,NaN,NaN,NaN
1172,James A. Michener.,1978-07-16,1,1,CHESAPEAKE,2,James A. Michener.**CHESAPEAKE,NaN,NaN,NaN
1280,James A. Michener.,1979-01-07,1,1,CHESAPEAKE,21,James A. Michener.**CHESAPEAKE,NaN,NaN,NaN
1506,James A. Michener.,1978-11-26,2,2,CHESAPEAKE,15,James A. Michener.**CHESAPEAKE,NaN,NaN,NaN
2316,James A. Michener.,1978-07-23,1,1,CHESAPEAKE,3,James A. Michener.**CHESAPEAKE,NaN,NaN,NaN
2501,James A. Michener.,1979-06-03,7,7,CHESAPEAKE,42,James A. Michener.**CHESAPEAKE,NaN,NaN,NaN


In [720]:
isbns1 = pd.concat((google_isbns_found[google_isbns_found.notnull()], isbns_1974_2008), axis = 0)

In [745]:
books_scraped = pd.merge(scraped_format2_df, isbns1, how = 'left', on = 'author_title')

In [746]:
books_scraped.drop(['isbn10_x', 'isbn13_x'], axis = 1, inplace = True)
books_scraped.rename(columns = lambda x: x.replace('_y', ''), inplace = True)

In [747]:
books_scraped = books_scraped.reindex_axis(sorted(books_scraped.columns), axis = 1)
books_scraped = books_scraped[books_scraped.isbn13.notnull()]

In [748]:
books_scraped.drop('asin_x', axis = 1, inplace = True)

In [750]:
books_scraped.reset_index(inplace = True, drop = True)

In [752]:
books_scraped.to_csv('data/1974-2008-metadata.tsv', sep = '\t', encoding = 'utf-8')

In [753]:
book_lists1 = pd.read_csv('data/1950-1974-metadata.tsv', delimiter = '\t', index_col=0)

In [756]:
book_lists1.head(1)

,asin,author,author_title,isbn10,isbn13,list_date,num_wks_on_list,rank_last_wk,rank_this_wk,title
0,,Thomas B. Costain,Thomas B. Costain**BELOW THE SALT,038504884X,9780385048842,1958-01-26,18,3,4,BELOW THE SALT


In [761]:
book_lists2 = pd.read_csv('data/1974-2008-metadata.tsv', delimiter = '\t', index_col=0)

In [764]:
book_lists2 = book_lists2.reindex_axis(sorted(book_lists2.columns), axis = 1)

In [766]:
book_lists2.to_csv('data/1974-2008-metadata.tsv', sep = '\t', encoding = 'utf-8')

In [768]:
book_lists2.head(1)

,asin,author,author_title,isbn10,isbn13,list_date,num_wks_on_list,rank_last_wk,rank_this_wk,title
0,NaN,Robin Cook.,Robin Cook.**SPHINX,NaN,9788422612742,1979-06-24,2,13,8,SPHINX


In [769]:
book_lists3 = pd.read_csv('data/2008-2011-metadata.tsv', delimiter='\t', index_col=0)

In [771]:
book_lists3.head(1)

,asin,author,author_title,isbn10,isbn13,list_date,num_wks_on_list,rank_last_wk,rank_this_wk,title
0,NaN,David Baldacci,David Baldacci**HELL'S CORNER,0446195529,9780446195522,2011-01-16,10,15,15,HELL'S CORNER


In [772]:
book_lists4 = pd.read_csv('data/2011-present-metadata.tsv', delimiter = '\t', index_col=0)

In [780]:
book_lists4[book_lists4.notnull()]

,asin,author,author_title,isbn10,isbn13,list_date,num_wks_on_list,rank_last_wk,rank_this_wk,title
0,NaN,Michael Prescott,Michael Prescott**STEALING FACES,None,A00B00547KH66,2011-08-27,0,0,20,STEALING FACES
1,NaN,Johanna Lindsey,Johanna Lindsey**HEARTS AFLAME,NaN,9780062106544,2011-08-27,1,0,15,HEARTS AFLAME
2,NaN,Chris Culver,Chris Culver**THE ABBEY,None,A00B004M18ZRS,2011-08-27,1,0,7,THE ABBEY
3,NaN,John Grisham,John Grisham**THE CONFESSION,0440245117,9780440245117,2011-08-27,10,9,14,THE CONFESSION
4,NaN,Sara Gruen,Sara Gruen**WATER FOR ELEPHANTS,1565125851,9781565125858,2011-08-27,31,10,4,WATER FOR ELEPHANTS
5,NaN,J A Jance,J A Jance**UNTIL PROVEN GUILTY,0061758221,9780061758225,2011-08-27,4,11,12,UNTIL PROVEN GUILTY
6,NaN,Lee Child,Lee Child**SECOND SON,None,9780345529725,2011-08-27,2,2,3,SECOND SON
7,NaN,Tatiana de Rosnay,Tatiana de Rosnay**SARAH’S KEY,1429985216,9781429985215,2011-08-27,6,4,9,SARAH’S KEY
8,NaN,CJ Lyons,CJ Lyons**BLIND FAITH,None,A00B0043M6J92,2011-08-27,1,0,2,BLIND FAITH
9,NaN,Nicholas Sparks,Nicholas Sparks**SAFE HAVEN,0446547603,9780446547604,2011-08-27,0,0,19,SAFE HAVEN


In [800]:
all_books1 = pd.concat((book_lists1, book_lists2, book_lists3, book_lists4), axis = 0)

In [801]:
all_books1

,asin,author,author_title,isbn10,isbn13,list_date,num_wks_on_list,rank_last_wk,rank_this_wk,title
0,,Thomas B. Costain,Thomas B. Costain**BELOW THE SALT,038504884X,9780385048842,1958-01-26,18,3,4,BELOW THE SALT
1,B01BH7CLA6,Elizabeth Goudge,Elizabeth Goudge**THE WHITE WITCH,0515041602,9780515041606,1958-01-26,3,7,6,THE WHITE WITCH
2,,Richard Mason,Richard Mason**THE WORLD OF SUZIE WONG,,,1958-01-26,27,15,12,THE WORLD OF SUZIE WONG
3,,Mary Ellen Chase,Mary Ellen Chase**THE EDGE OF DARKNESS,,,1958-01-26,12,9,8,THE EDGE OF DARKNESS
4,B0067F6WWM,James Jones,James Jones**SOME CAME RUNNING,0440182611,9780440182610,1958-01-26,3,4,5,SOME CAME RUNNING
5,,Paul Gallico,Paul Gallico**THOMASINA,0930330935,9780930330934,1958-01-26,7,13,14,THOMASINA
6,B004P1JFIS,Grace Metalious,Grace Metalious**PEYTON PLACE,1555534007,9781555534004,1958-01-26,72,14,16,PEYTON PLACE
7,,James Gould Cozzens,James Gould Cozzens**BY LOVE POSSESSED,0786705035,9780786705030,1958-01-26,23,1,1,BY LOVE POSSESSED
8,B003V8B5XO,Ayn Rand,Ayn Rand**ATLAS SHRUGGED,0452011876,9780452011878,1958-01-26,18,6,9,ATLAS SHRUGGED
9,B0035JEPAO,Nevil Shute,Nevil Shute**ON THE BEACH,1842322761,9781842322765,1958-01-26,27,11,15,ON THE BEACH


In [791]:
isbns_still_missing_combined = all_books[all_books.isbn13.isnull()]['author_title'].unique().tolist()

In [792]:
isbns_still_missing_combined.extend(all_books[all_books.isbn13 == ' ']['author_title'].unique().tolist())

In [795]:
isbns_still_missing_combined

['Max Schulman**RALLY ROUND THE FLAG BOYS!',
 'Francis Van Wyck Mason**GOLDEN ADMIRAL',
 'Paul Hyde Bonner**HOTEL TALLYRAND',
 'Ross MacDonald**THE GOODBYE LOOK',
 'A.A. Milnei**WINNIE ILLE PU',
 'Angela Mackail Thirkell**HAPPY RETURN',
 'Giovanni Guareschi**DON CAMILLO AND HIS FLOCK',
 'Robert Reynolds**THE SINNER OF SAINT AMBROSE',
 'Francis Van Wyck Mason**HIMALAYAN ASSIGNMENT',
 'Harold Lenoir Davis**WINDS OF MORNING',
 'William E. Barrett**THE SHADOWS OF THE IMAGES',
 'A.B. Guthrie Jr**THESE THOUSAND HILLS',
 'Louis Bromfield**MR.SMITH',
 'Lin Yutang**THE VERMILLION GATE',
 'Leon Uris**ARMEGEDDON',
 'Inglis Clark Fletcher**THE SCOTSWOMAN',
 'Alice Tisdale Hobart**VENTURE INTO DARKNESS',
 'Theodore Bonnet**DUTCH',
 'John Phillips Marquand**SINCERELY, WILLIS WAYDE',
 'Eleanor Palffy**THE LADY AND THE PAINTER',
 'Robert Gover**$100 MISUNDERSTANDING',
 'Edward Gilbert**SILVER SPOON',
 'Angela Thirkell**COUNTRY CHRONICLE',
 'Harnett T. Kane**PATHWAY TO THE STARS',
 'Susan Yorke**THE WI

In [796]:
with open('data/isbns_still_missing.txt', 'w') as f:
    for book in isbns_still_missing_combined:
        f.write("%s\n" % book)

In [797]:
all_books = all_books[all_books.isbn13.notnull()]

In [798]:
all_books = all_books[all_books.isbn13 != ' ']

In [803]:
all_books.to_csv('data/all_book_lists_combined.tsv', sep = '\t', encoding = 'utf-8')

In [829]:
from datetime import timedelta

In [878]:
start_time = datetime.now()

with open("data/first_100000_book_rows.json", 'r') as f:
#     a = []
    for line in f:
        for isbn in test_list:
            if isbn in line:
                if not os.path.isfile('test/search_reviews_test.json'):
#                     a.append(line.encode('utf-8'))
                    with open('test/search_reviews_test.json', mode='w') as f:
                        f.write(json.dumps(line.encode('utf-8'), indent=2))
                else:
#                     with open('test/search_reviews_test.json') as f:
#                         feeds = json.load(f)
# #                         print feeds
#                     feeds.append(line.encode('utf-8'))
                    with open('test/search_reviews_test.json', mode='a') as f:
                        f.write(json.dumps(line.encode('utf-8'), indent=2))
end_time = datetime.now()
print datetime.strftime(start_time, '%H:%M:%S.%f')
print datetime.strftime(end_time, '%H:%M:%S.%f')

print("Reading of {} lines took {} seconds".format(100000, (end_time - start_time).seconds))

09:29:09.673781
09:29:12.122134
Reading of 100000 lines took 2 seconds


In [873]:
pwd

u'/Users/asalmitrofanov/DSI/fiction-market-analysis-capstone'

In [804]:
test_list = ['000100039X', '0007444117', '0007442920', '0007419139', '0007228023', '1466850604', '0440245117']

In [847]:
all_book_isbns13 = all_books.isbn13.unique().tolist()
all_book_isbns10 = all_books.isbn10.unique().tolist()

In [913]:
with open('data/all_books_isbn13.txt', 'w') as f:
    for isbn in all_book_isbns13:
        f.write("%s\n" % isbn)

In [848]:
all_book_asins = all_books.asin.unique().tolist()

In [850]:
all_book_asins.remove(' ')

In [857]:
all_book_isbns10.remove(' ')

In [859]:
all_book_isbns10 = [x for x in all_book_isbns10 if x != None]

In [865]:
all_book_asins = [x for x in all_book_asins if x != None]

In [867]:
all_book_isbns10

['038504884X',
 '0515041602',
 '0440182611',
 '0930330935',
 '1555534007',
 '0786705035',
 '0452011876',
 '1842322761',
 '0375701230',
 '068566144X',
 '0451015789',
 '0553235206',
 '0312033567',
 '0316290602',
 '0671814524',
 '0940322196',
 '0451008685',
 '0394756967',
 '1583220089',
 '0718208110',
 '0316253502',
 '1564782603',
 '1556526016',
 '0881841412',
 '0316626597',
 '0767908198',
 '0304352411',
 '0340026065',
 '0316955132',
 '9997409124',
 '0452269563',
 '0140123466',
 '0316181749',
 '1559210354',
 '1568582463',
 '0892440384',
 '0394448146',
 '0143039024',
 '006013495X',
 '0812810147',
 '0451194004',
 '8203847080',
 '0310241634',
 '0061145394',
 '0679736638',
 '0449213447',
 '9997406249',
 '0340105267',
 '0030624401',
 '0316955108',
 '0829423508',
 '1844080366',
 '0671753282',
 '0877956537',
 '006075186X',
 '0671827634',
 '0316767727',
 '8493388335',
 '1842322540',
 '0571203132',
 '9997403479',
 '0142000655',
 '044631076X',
 '1557501734',
 '0285636049',
 '9997410602',
 '00605293

In [868]:
isbns = all_book_isbns10

In [869]:
isbns.extend(all_book_asins)

In [871]:
with open('data/isbns.txt', 'w+') as f:
    for isbn in isbns:
        f.write("%s\n" % isbn)

In [823]:
with open('data/first_100_rows.json', 'r') as f:
    data = []
    for l in f:
        data.append(json.loads(l))

In [824]:
data

[{u'asin': u'0000000116',
  u'helpful': [5, 5],
  u'overall': 4.0,
  u'reviewText': u"Interesting Grisham tale of a lawyer that takes millions of dollars from his firm after faking his own death. Grisham usually is able to hook his readers early and ,in this case, doesn't play his hand to soon. The usually reliable Frank Mueller makes this story even an even better bet on Audiobook.",
  u'reviewTime': u'04 27, 2002',
  u'reviewerID': u'AH2L9G3DQHHAJ',
  u'reviewerName': u'chris',
  u'summary': u'Show me the money!',
  u'unixReviewTime': 1019865600},
 {u'asin': u'0000000116',
  u'helpful': [0, 0],
  u'overall': 1.0,
  u'reviewText': u'The thumbnail is a shirt.  The product shown is a shoe.  The description is a book.This reviewer is confused.',
  u'reviewTime': u'03 24, 2014',
  u'reviewerID': u'A2IIIDRK3PRRZY',
  u'reviewerName': u'Helene',
  u'summary': u'Listing is all screwed up',
  u'unixReviewTime': 1395619200},
 {u'asin': u'0000000868',
  u'helpful': [10, 10],
  u'overall': 4.0,


In [ ]:
import json
import time

start_time = time.now()
loop_num = 100000
with open('data/book_reviews.json') as f:
    for book in books:
        for _  in range(loop_num)
            entry = f.read_next_line()
            entry = json.loads(line)
            if entry['asin'] == mybook['isbn10'] or entry['asin'] == mybook['isbn13'] or entry['asin'] == mybook['asin']:
                pass

end_time = time.now()

print("Reading of {} took {} seconds".format(loop_num, (end_time - start_time).seconds)

In [ ]:
from future import print_function

lines_to_read = 10
with open("datafile") as myfile:
    head = (next(myfile) for x in range(lines_to_read))
    for line in head:
        print(line, end="")

In [897]:
from collections import defaultdict, Counter

In [882]:
with open("data/first_100000_book_rows.json", 'r') as f:
    reviews = (json.loads(line) for line in f)

In [900]:
with open("data/first_100000_book_rows.json", 'r') as f:
    reviews = (json.loads(line) for line in f)
#     list_of_reviews = defaultdict(list)
    i = 0
    j = 0
    for line in reviews:
        i += 1
        if line['asin'] in isbns:
            j += 1
#             print type(line)
#             list_of_reviews[line['asin']].append(line)
            if not os.path.isfile('data/join_reviews_test.json'):
#                     a.append(line.encode('utf-8'))
                with open('data/join_reviews_test.json', mode='w') as f:
                    f.write(json.dumps(line, indent = 2))
            else:
#                     with open('test/search_reviews_test.json') as f:
#                         feeds = json.load(f)
# #                         print feeds
#                     feeds.append(line.encode('utf-8'))
                with open('data/join_reviews_test.json', mode='a') as f:
                    f.write(json.dumps(line, indent = 2))
        if i % 10000 == 0:
            print '{} lines checked. {} lines written.'.format(i, j)
#             with open('data/join_reviews.json', 'a') as f:
#                 f.write(line)

10000 lines checked. 0 lines written.
20000 lines checked. 273 lines written.
30000 lines checked. 273 lines written.
40000 lines checked. 273 lines written.
50000 lines checked. 273 lines written.
60000 lines checked. 273 lines written.
70000 lines checked. 273 lines written.
80000 lines checked. 273 lines written.
90000 lines checked. 273 lines written.
100000 lines checked. 273 lines written.


In [889]:
list_of_reviews

defaultdict(list,
            {u'000612609X': [{u'asin': u'000612609X',
               u'helpful': [0, 0],
               u'overall': 5.0,
               u'reviewText': u'A must read for all who love Viking sagas.A very good book, sometime slow but it reads smothly indeed',
               u'reviewTime': u'05 12, 2014',
               u'reviewerID': u'A13TSLDBD5Q9X8',
               u'reviewerName': u'Alessandro',
               u'summary': u'Brilliant',
               u'unixReviewTime': 1399852800},
              {u'asin': u'000612609X',
               u'helpful': [1, 1],
               u'overall': 5.0,
               u'reviewText': u"If you are a guy with testicles, you will like this book. The epic describes the adventures of a Viking, Orm, from his youth up until his last expedition. He never stays in one place for too long and is pretty much on the search for blood the the span of his lifetime. The only drawback to this book is the prattling priest, who tries to soften and dull the

In [907]:
with open('data/isbns.txt', 'r') as i:
        isbns = [l.strip() for l in i]

In [910]:
with open("data/first_100000_book_rows.json", 'r') as input_file:
    reviews = (json.loads(line) for line in input_file)
    i = 0
    j = 0
    with open('data/isbns.txt', 'r') as isbns:
        list_of_isbns = [l.strip() for l in isbns]
    for line in reviews:
        i += 1
        if line['asin'] in list_of_isbns:
            j += 1
            if not os.path.isfile('data/join_reviews_test.json'):
                with open('data/join_reviews_test.json', mode='w') as output_file:
                    output_file.write(json.dumps(line, indent = 2))
            else:
                with open('data/join_reviews_test.json', mode='a') as output_file:
                    output_file.write(json.dumps(line, indent = 2))
        if i % 10000 == 0:
            print '{} lines checked. {} lines written.'.format(i, j)


10000 lines checked. 0 lines written.
20000 lines checked. 273 lines written.
30000 lines checked. 273 lines written.
40000 lines checked. 273 lines written.
50000 lines checked. 273 lines written.
60000 lines checked. 273 lines written.
70000 lines checked. 273 lines written.
80000 lines checked. 273 lines written.
90000 lines checked. 273 lines written.
100000 lines checked. 273 lines written.
